In [ ]:
import torch

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("CUDA is not installed or not available.")

CUDA Version: 11.8


In [ ]:
import urllib
import cv2
import numpy as np
import time
from inference import get_model
import urllib

# Initialize the model
model = get_model(model_id="merged-with-extra/2", api_key="ZKyTobIOpOugXJsa4i8l")

url = 'http://192.168.1.146/cam-hi.jpg'

# Set the desired frame rate
fps = 6

frame_interval = 1 / fps  # Time in seconds between frames (0.1 seconds for 10 FPS)


[12/02/24 05:29:35] WARNING  Your inference package version 0.28.1 is out of date! Please upgrade to ]8;id=585790;file://c:\Users\Nattakorn\anaconda3\envs\computer_vision\lib\site-packages\inference\core\__init__.py\__init__.py]8;;\:]8;id=478870;file://c:\Users\Nattakorn\anaconda3\envs\computer_vision\lib\site-packages\inference\core\__init__.py#41\41]8;;\
                             version 0.29.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

In [ ]:
import cv2
import time
import requests
import numpy as np
from inference import get_model

# ESP32 endpoint
esp32_url = 'http://192.168.199.162/send'  # Update with your ESP32 endpoint
model = get_model(model_id="merged-with-extra/2", api_key="ZKyTobIOpOugXJsa4i8l")

# Initialize video capture
cap = cv2.VideoCapture(0)  # Use camera 0 (default camera)

# Set the desired frame rate
fps = 5
frame_interval = 1 / fps  

# Dictionary to track class counts
class_counts = {}

# Timer for sending predictions every 1 second
last_sent_time = time.time()

while cap.isOpened():
    t0 = time.time()
    
    # Capture frame from the camera
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame from the camera.")
        break

    # Run inference
    try:
        _, img_bytes = cv2.imencode('.jpg', frame)
        img = img_bytes.tobytes()
        results = model.infer(
            image=img,
            confidence=0.3,
            iou_threshold=0.5
        )
    except Exception as e:
        print(f"Error during inference: {e}")
        results = []  # Ensure `results` is valid even on failure

    # Prepare predictions list
    predictions_list = []

    for r in results:
        for prediction in r.predictions:
            # Get bounding box coordinates
            x, y, w, h = int(prediction.x), int(prediction.y), int(prediction.width), int(prediction.height)
            
            # Update class count
            class_name = prediction.class_name
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
            
            # Append to predictions list
            predictions_list.append({
                "class": class_name,
                "x": x,
                "y": y,
                "width": w,
                "height": h,
                "confidence": prediction.confidence
            })

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Check if 1 second has passed since the last send
    current_time = time.time()
    if current_time - last_sent_time >= 3:
        # Send predictions (even if empty) to ESP32
        print(predictions_list)
        try:
            response = requests.post(esp32_url, json=predictions_list, timeout=5)
            print(f"Predictions sent to ESP32: {predictions_list}, Response: {response.status_code}")
        except Exception as e:
            print(f"Error sending predictions to ESP32: {e}")
        
        # Update the last sent time
        last_sent_time = current_time

    # Display the frame with bounding boxes
    cv2.imshow('Camera Feed', frame)

    # Wait for the key press and exit on 'q'
    if ord('q') == cv2.waitKey(1):
        break

    # Control the FPS by sleeping for the frame interval
    elapsed_time = time.time() - t0
    if elapsed_time < frame_interval:
        time.sleep(frame_interval - elapsed_time)

# Release resources and close all windows
cap.release()
cv2.destroyAllWindows()


[12/02/24 12:29:18] WARNING  Your inference package version 0.28.1 is out of date! Please upgrade to ]8;id=95661;file://c:\Users\Nattakorn\anaconda3\envs\computer_vision\lib\site-packages\inference\core\__init__.py\__init__.py]8;;\:]8;id=318006;file://c:\Users\Nattakorn\anaconda3\envs\computer_vision\lib\site-packages\inference\core\__init__.py#41\41]8;;\
                             version 0.29.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

[{'class': 'Pepsi', 'x': 381, 'y': 245, 'width': 122, 'height': 240, 'confidence': 0.8867650628089905}, {'class': 'Coca-Cola', 'x': 147, 'y': 237, 'width': 97, 'height': 221, 'confidence': 0.8795159459114075}]
Error sending predictions to ESP32: HTTPConnectionPool(host='192.168.199.162', port=80): Read timed out. (read timeout=5)
[{'class': 'Pepsi', 'x': 381, 'y': 245, 'width': 122, 'height': 240, 'confidence': 0.8867650628089905}, {'class': 'Coca-Cola', 'x': 147, 'y': 237, 'width': 97, 'height': 221, 'confidence': 0.8795159459114075}]
Error sending predictions to ESP32: HTTPConnectionPool(host='192.168.199.162', port=80): Read timed out. (read timeout=5)
[{'class': 'Pepsi', 'x': 381, 'y': 245, 'width': 122, 'height': 240, 'confidence': 0.8867650628089905}, {'class': 'Coca-Cola', 'x': 147, 'y': 237, 'width': 97, 'height': 221, 'confidence': 0.8795159459114075}]
Predictions sent to ESP32: [{'class': 'Pepsi', 'x': 381, 'y': 245, 'width': 122, 'height': 240, 'confidence': 0.88676506280899

KeyboardInterrupt: 

In [1]:
import cv2
import time
import requests
import numpy as np
import urllib.request
from inference import get_model

# ESP32 endpoint
esp32_url = 'http://192.168.199.162/send'  # Update with your ESP32 endpoint
model = get_model(model_id="merged-with-extra/2", api_key="ZKyTobIOpOugXJsa4i8l")

# Image URL
url = 'http://192.168.199.31/cam-hi.jpg'

# Set the desired frame rate
fps = 5
frame_interval = 1 / fps  

# Dictionary to track class counts
class_counts = {}

# Timer for sending predictions every 3 seconds
last_sent_time = time.time()

In [ ]:
# import cv2
# import time
# import requests
# import numpy as np
# import urllib.request
# from inference import get_model

# # ESP32 endpoint
# esp32_url = 'http://192.168.199.162/send'  # Update with your ESP32 endpoint
# model = get_model(model_id="merged-with-extra/2", api_key="ZKyTobIOpOugXJsa4i8l")

# # Image URL
# url = 'http://192.168.199.31/cam-hi.jpg'

# # Set the desired frame rate
# fps = 5
# frame_interval = 1 / fps  

# # Dictionary to track class counts
# class_counts = {}

# # Timer for sending predictions every 3 seconds
# last_sent_time = time.time()

while True:
    t0 = time.time()
    
    # Capture frame from URL
    try:
        img_resp = urllib.request.urlopen(url)
        img_np = np.array(bytearray(img_resp.read()), dtype=np.uint8)
        frame = cv2.imdecode(img_np, -1)
        frame = cv2.flip(frame, 0)
    except Exception as e:
        print(f"Error fetching image from URL: {e}")
        continue

    # Run inference
    try:
        _, img_bytes = cv2.imencode('.jpg', frame)
        img = img_bytes.tobytes()
        results = model.infer(
            image=img,
            confidence=0.5,
            iou_threshold=0.5
        )
    except Exception as e:
        print(f"Error during inference: {e}")
        results = []  # Ensure `results` is valid even on failure

    # Prepare predictions list
    predictions_list = []

    for r in results:
        for prediction in r.predictions:
            # Get bounding box coordinates
            x, y, w, h = int(prediction.x), int(prediction.y), int(prediction.width), int(prediction.height)
            
            # Update class count
            class_name = prediction.class_name
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
            
            # Append to predictions list
            predictions_list.append({
                "class": class_name,
                "x": x,
                "y": y,
                "width": w,
                "height": h,
                "confidence": prediction.confidence
            })

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Check if 3 seconds have passed since the last send
    current_time = time.time()
    if current_time - last_sent_time >= 3:
        # Send predictions (even if empty) to ESP32
        print(predictions_list)
        try:
            response = requests.post(esp32_url, json=predictions_list, timeout=5) 
            res_web = requests.post("https://double-table.vercel.app/api/mongo-insert", json=predictions_list, timeout=5) #https://double-table.vercel.app/api/mongo-insert
            print(f"Predictions sent to ESP32: {predictions_list}, Response: {response.status_code}")
            print(f"web app: {predictions_list}, Response: {response.status_code}")
        except Exception as e:
            print(f"Error sending predictions to ESP32: {e}")
        
        # Update the last sent time
        last_sent_time = current_time

    # Display the frame with bounding boxes
    cv2.imshow('Camera Feed', frame)

    # Wait for the key press and exit on 'q'
    if ord('q') == cv2.waitKey(1):
        break

    # Control the FPS by sleeping for the frame interval
    elapsed_time = time.time() - t0
    if elapsed_time < frame_interval:
        time.sleep(frame_interval - elapsed_time)

# Release resources and close all windows
cv2.destroyAllWindows()


[{'class': 'Pepsi', 'x': 359, 'y': 277, 'width': 121, 'height': 329, 'confidence': 0.8460897207260132}]
Predictions sent to ESP32: [{'class': 'Pepsi', 'x': 359, 'y': 277, 'width': 121, 'height': 329, 'confidence': 0.8460897207260132}], Response: 200
web app: [{'class': 'Pepsi', 'x': 359, 'y': 277, 'width': 121, 'height': 329, 'confidence': 0.8460897207260132}], Response: 200
[{'class': 'Pepsi', 'x': 360, 'y': 272, 'width': 120, 'height': 319, 'confidence': 0.8442016839981079}]
Predictions sent to ESP32: [{'class': 'Pepsi', 'x': 360, 'y': 272, 'width': 120, 'height': 319, 'confidence': 0.8442016839981079}], Response: 200
web app: [{'class': 'Pepsi', 'x': 360, 'y': 272, 'width': 120, 'height': 319, 'confidence': 0.8442016839981079}], Response: 200
[{'class': 'Pepsi', 'x': 359, 'y': 275, 'width': 119, 'height': 323, 'confidence': 0.8612132668495178}]
Predictions sent to ESP32: [{'class': 'Pepsi', 'x': 359, 'y': 275, 'width': 119, 'height': 323, 'confidence': 0.8612132668495178}], Response